In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import tensorflow as tf
from notebooks.create_spectrogram import (
    create_spectrograms_from_audio_dataset,
    extract_patches_from_audio_dataset 
)
from notebooks.helper_functions import (
    convert_prefetchdataset_to_numpy_arrays,
    lite_model_from_file_predicts_dataset,
    find_decision_threshold,
    convert_bytes,
    get_file_size,
    evaluate_prediction_with_threshold
    )

2024-09-22 16:06:31.251023: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-22 16:06:31.337004: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-22 16:06:31.337041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-22 16:06:31.343739: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-22 16:06:31.368710: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-22 16:06:31.369985: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
test_dataset = tf.keras.utils.audio_dataset_from_directory("../../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=False, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=False, seed=3407)

time_ser_x_val_np, time_ser_y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
time_ser_x_test_np, time_ser_y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

Found 1393 files belonging to 2 classes.


2024-09-22 16:06:35.727025: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-22 16:06:35.727539: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-09-22 16:06:35.872162: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-09-22 16:06:35.894348: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1380 files belonging to 2 classes.


In [3]:
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = 16000) #.cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = 16000) #.cache().prefetch(tf.data.AUTOTUNE)

spec_x_val_np, spec_y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
spec_x_test_np, spec_y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [4]:
test_patches_ds = extract_patches_from_audio_dataset(test_dataset, sample_rate = 16000, patch_size=8) #.cache().prefetch(tf.data.AUTOTUNE)
val_patches_ds = extract_patches_from_audio_dataset(val_dataset, sample_rate = 16000, patch_size=8) #.cache().prefetch(tf.data.AUTOTUNE)

patch_x_test_np, patch_y_test_np = convert_prefetchdataset_to_numpy_arrays(test_patches_ds, data_type="not spectrogram")
patch_x_val_np, patch_y_val_np = convert_prefetchdataset_to_numpy_arrays(val_patches_ds, data_type="not spectrogram")

In [17]:
tflite_models = [
    ("../../models/spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_spec.tflite", "CNN-Mel"), 
    ("../../models/spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz.tflite", "SqueezeNet-Mel"), 
    ("../../models/spectrogram_models_from_notebooks/bnn/hpo/bnn_mel_spec_tf_lite.tflite", "BNN-Mel"),
    ('../../models/spectrogram_models_from_notebooks/vit/quantized/vit_mel_spec.tflite', "Transformer-Mel"),
    ("../../models/time_series_models_from_notebooks/cnn/quantized/cnn_time_series.tflite", "CNN-Time"), 
    ("../../models/time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz.tflite", "SqueezeNet-Time"), 
    ("../../models/time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_tf_lite.tflite", "BNN-Time"),
    ("../../models/time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series.tflite", "Transformer-Time"),
          ]

float16_models = [
    ("../../models/spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_spec_float16q.tflite", "CNN-Mel"), 
    ("../../models/spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_float16q.tflite", "SqueezeNet-Mel"), 
    ("../../models/spectrogram_models_from_notebooks/bnn/hpo/bnn_mel_spec_float16q.tflite", "BNN-Mel"),
    ('../../models/spectrogram_models_from_notebooks/vit/quantized/vit_mel_spec_float16q.tflite', "Transformer-Mel"),
    ("../../models/time_series_models_from_notebooks/cnn/quantized/cnn_time_series_float16q.tflite", "CNN-Time"), 
    ("../../models/time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_float16q.tflite", "SqueezeNet-Time"), 
    ("../../models/time_series_models_from_notebooks/bnn/hpo/bnn_time_series_float16q.tflite", "BNN-Time"),
    ("../../models/time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_float16q.tflite", "Transformer-Time"),
          ]

full_int_models = [
    # ("../../models/spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_spec_full_int_q.tflite", "CNN-Mel"), 
    # ("../../models/spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_full_int_q.tflite", "SqueezeNet-Mel"), 
    # ("../../models/spectrogram_models_from_notebooks/bnn/hpo/bnn_mel_spec_full_int_q.tflite", "BNN-Mel"),
    # ('../../models/spectrogram_models_from_notebooks/vit/quantized/vit_mel_spec_full_int_q.tflite', "Transformer-Mel"),
    # ("../../models/time_series_models_from_notebooks/cnn/quantized/cnn_time_series_full_int_q.tflite", "CNN-Time"), 
    # ("../../models/time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_full_int_q.tflite", "SqueezeNet-Time"), 
    # ("../../models/time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_full_int_q.tflite", "BNN-Time"),
    ("../../models/time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_full_int_q.tflite", "Transformer-Time"),
          ]

In [22]:
def calculate_metrics(model_data, thresholds_and_metrics_dict):
    print(model_data[1])
    if model_data[1].endswith("Mel"):
        if model_data[1].startswith("Transformer"):
            x_val = patch_x_val_np
            x_test = patch_x_test_np
        else:
            x_val = spec_x_val_np
            x_test = spec_x_test_np
    elif model_data[1].endswith("Time"):
        x_val = time_ser_x_val_np
        x_test = time_ser_x_test_np
    
    # Get threshold using val dataset
    _, y_pred_prob_val = lite_model_from_file_predicts_dataset(model_data[0], x_val)
    threshold = find_decision_threshold(time_ser_y_val_np, y_pred_prob_val, model_format="tflite")
    
    # Predict test dataset using threshold
    _, y_pred_prob_test = lite_model_from_file_predicts_dataset(model_data[0], x_test)
    test_metrics_dict = evaluate_prediction_with_threshold(time_ser_y_test_np, y_pred_prob_test, model_format="tflite", optimal_threshold=threshold)

    file_size = convert_bytes(get_file_size(model_data[0]), "KB")
    print("\n")
    
    thresholds_and_metrics_dict[model_data[1]] = {"threshold": threshold, **test_metrics_dict, **{"file size": file_size}}

#### TFLite models

In [23]:
tflite_models_thresholds_and_metrics = {}

for model_data in tflite_models: 
    calculate_metrics(model_data, tflite_models_thresholds_and_metrics)

CNN-Mel
Accuracy: 98.71%
Recall: 98.25%
Precision: 97.83%
F1-score: 98.04%
F2-score: 98.17%
File size: 103.875 Kilobytes


SqueezeNet-Mel
Accuracy: 98.56%
Recall: 99.34%
Precision: 96.40%
F1-score: 97.85%
F2-score: 98.74%
File size: 2856.07 Kilobytes


BNN-Mel
Accuracy: 95.62%
Recall: 95.85%
Precision: 91.27%
F1-score: 93.50%
F2-score: 94.90%
File size: 157.906 Kilobytes


Transformer-Mel
Accuracy: 97.49%
Recall: 96.72%
Precision: 95.68%
F1-score: 96.20%
F2-score: 96.51%
File size: 154.801 Kilobytes


CNN-Time
Accuracy: 98.64%
Recall: 96.94%
Precision: 98.89%
F1-score: 97.91%
F2-score: 97.33%
File size: 44.035 Kilobytes


SqueezeNet-Time
Accuracy: 96.20%
Recall: 97.16%
Precision: 91.75%
F1-score: 94.38%
F2-score: 96.03%
File size: 150.871 Kilobytes


BNN-Time
Accuracy: 83.20%
Recall: 87.77%
Precision: 69.31%
F1-score: 77.46%
F2-score: 83.33%
File size: 44.512 Kilobytes


Transformer-Time
Accuracy: 95.19%
Recall: 97.16%
Precision: 89.18%
F1-score: 93.00%
F2-score: 95.45%
File size: 43.2

In [24]:
print("TFLite models results with calibrated optimal thresholds: ")
pd.DataFrame(tflite_models_thresholds_and_metrics).T

TFLite models results with calibrated optimal thresholds: 


,threshold,Accuracy,Precision,Recall,F1-score,F2-score,file size
CNN-Mel,0.728124,0.99,0.98,0.98,0.98,0.98,103.875 Kilobytes
SqueezeNet-Mel,0.734869,0.99,0.96,0.99,0.98,0.99,2856.07 Kilobytes
BNN-Mel,0.400291,0.96,0.91,0.96,0.94,0.95,157.906 Kilobytes
Transformer-Mel,-0.118999,0.97,0.96,0.97,0.96,0.97,154.801 Kilobytes
CNN-Time,0.450375,0.99,0.99,0.97,0.98,0.97,44.035 Kilobytes
SqueezeNet-Time,0.332308,0.96,0.92,0.97,0.94,0.96,150.871 Kilobytes
BNN-Time,0.306664,0.83,0.69,0.88,0.77,0.83,44.512 Kilobytes
Transformer-Time,0.334944,0.95,0.89,0.97,0.93,0.95,43.238 Kilobytes


#### Float16 models

In [25]:
float16_models_thresholds_and_metrics = {}

for model_data in float16_models: 
    calculate_metrics(model_data, float16_models_thresholds_and_metrics)

CNN-Mel
Accuracy: 98.78%
Recall: 98.25%
Precision: 98.04%
F1-score: 98.15%
F2-score: 98.21%
File size: 55.07 Kilobytes


SqueezeNet-Mel
Accuracy: 98.56%
Recall: 99.34%
Precision: 96.40%
F1-score: 97.85%
F2-score: 98.74%
File size: 1442.977 Kilobytes


BNN-Mel
Accuracy: 95.48%
Recall: 95.63%
Precision: 91.06%
F1-score: 93.29%
F2-score: 94.68%
File size: 85.387 Kilobytes


Transformer-Mel
Accuracy: 97.49%
Recall: 96.72%
Precision: 95.68%
F1-score: 96.20%
F2-score: 96.51%
File size: 93.586 Kilobytes


CNN-Time
Accuracy: 98.64%
Recall: 96.94%
Precision: 98.89%
F1-score: 97.91%
F2-score: 97.33%
File size: 27.406 Kilobytes


SqueezeNet-Time
Accuracy: 96.20%
Recall: 97.16%
Precision: 91.75%
F1-score: 94.38%
F2-score: 96.03%
File size: 97.117 Kilobytes


BNN-Time
Accuracy: 82.91%
Recall: 87.99%
Precision: 68.77%
F1-score: 77.20%
F2-score: 83.33%
File size: 30.641 Kilobytes


Transformer-Time
Accuracy: 95.19%
Recall: 97.16%
Precision: 89.18%
F1-score: 93.00%
F2-score: 95.45%
File size: 33.258 K

In [26]:
pd.DataFrame(float16_models_thresholds_and_metrics).T

,threshold,Accuracy,Precision,Recall,F1-score,F2-score,file size
CNN-Mel,0.727961,0.99,0.98,0.98,0.98,0.98,55.07 Kilobytes
SqueezeNet-Mel,0.734827,0.99,0.96,0.99,0.98,0.99,1442.977 Kilobytes
BNN-Mel,0.400288,0.95,0.91,0.96,0.93,0.95,85.387 Kilobytes
Transformer-Mel,-0.118942,0.97,0.96,0.97,0.96,0.97,93.586 Kilobytes
CNN-Time,0.448911,0.99,0.99,0.97,0.98,0.97,27.406 Kilobytes
SqueezeNet-Time,0.332403,0.96,0.92,0.97,0.94,0.96,97.117 Kilobytes
BNN-Time,0.306683,0.83,0.69,0.88,0.77,0.83,30.641 Kilobytes
Transformer-Time,0.334774,0.95,0.89,0.97,0.93,0.95,33.258 Kilobytes


#### Full integer models

In [10]:
full_int_models_thresholds_and_metrics = {}

In [22]:
def run_full_int_q_tflite_model(tflite_file, indices, x_data):
  
  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions_prob = np.zeros((len(indices),), dtype=int)
  predictions_class = np.zeros((len(indices),), dtype=int)
  for i, test_image_index in enumerate(indices):
    test_data_point = x_data[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_data_point = test_data_point / input_scale + input_zero_point

    test_data_point = np.expand_dims(test_data_point, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_data_point)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    predictions_class[i] = output.argmax()
    predictions_prob[i] = output[1]

  return predictions_class, predictions_prob

def full_int_model_predict(tflite_file, x_data):
  indices = range(len(x_data))
  predictions = run_full_int_q_tflite_model(tflite_file, indices, x_data)
  return predictions

In [23]:
def full_int_model_calculate_metrics(model_data, thresholds_and_metrics_dict):
    print(model_data[1])
    if model_data[1].endswith("Mel"):
        if model_data[1].startswith("Transformer"):
            x_val = patch_x_val_np
            x_test = patch_x_test_np
        else:
            x_val = spec_x_val_np
            x_test = spec_x_test_np
    elif model_data[1].endswith("Time"):
        x_val = time_ser_x_val_np
        x_test = time_ser_x_test_np
    
    # Get threshold using val dataset
    _, y_pred_prob = full_int_model_predict(model_data[0], x_val)
    threshold = find_decision_threshold(time_ser_y_val_np, y_pred_prob, model_format="tflite")
    print(threshold)

    # Predict test dataset using threshold
    threshold = 21
    _, y_pred_prob = full_int_model_predict(model_data[0], x_test)
    test_metrics_dict = evaluate_prediction_with_threshold(time_ser_y_test_np, y_pred_prob, model_format="tflite", optimal_threshold=threshold)

    file_size = convert_bytes(get_file_size(model_data[0]), "KB")
    print(file_size)

    print("\nTime for Test dataset:")
    time_data = []
    for data_point in x_test:
        start_time = time.time()
        predictions = full_int_model_predict(model_data[0], [data_point])
        elapsed_time = time.time() - start_time
        time_data.append(elapsed_time)
    print("\nTime to make a prediction for a single data point")
    print(f"Mean: {round(np.mean(time_data), 3)} seconds")
    print(f"Max: {round(np.max(time_data), 3)} seconds")
    print(f"Min: {round(np.min(time_data), 3)} seconds")
    print(round(np.mean(time_data)*1000, 3), "ms")
    print("\n")
    
    thresholds_and_metrics_dict[model_data[1]] = {"threshold": threshold, **test_metrics_dict, **{"file size": file_size}, **{"latency": round(np.mean(time_data)*1000, 3)}}

In [24]:
for model_data in full_int_models: 
    full_int_model_calculate_metrics(model_data, full_int_models_thresholds_and_metrics)

Transformer-Time
19.0
Accuracy: 87.22%
Recall: 78.60%
Precision: 81.82%
F1-score: 80.18%
F2-score: 79.23%
File size: 29.352 Kilobytes
29.352 Kilobytes

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.007 seconds
Max: 0.014 seconds
Min: 0.006 seconds
7.223 ms




In [16]:
pd.DataFrame(full_int_models_thresholds_and_metrics).T

,threshold,Accuracy,Precision,Recall,F1-score,F2-score,file size,latency
CNN-Mel,218.0,0.99,0.98,0.98,0.98,0.98,30.125 Kilobytes,1.306
SqueezeNet-Mel,117.0,0.99,0.96,1.0,0.98,0.99,807.539 Kilobytes,10.91
BNN-Mel,102.0,0.95,0.91,0.96,0.93,0.95,49.953 Kilobytes,1.465
Transformer-Mel,185.0,0.96,0.94,0.93,0.94,0.93,60.836 Kilobytes,2.78
CNN-Time,29.0,0.97,0.95,0.95,0.95,0.95,18.906 Kilobytes,3.021
SqueezeNet-Time,128.0,0.79,0.61,1.0,0.75,0.88,87.039 Kilobytes,15.825
BNN-Time,205.0,0.73,0.55,0.97,0.7,0.84,25.172 Kilobytes,3.998
Transformer-Time,19.0,0.87,0.81,0.79,0.8,0.8,29.352 Kilobytes,7.297


In [46]:
pd.DataFrame(full_int_models_thresholds_and_metrics).T

,threshold,Accuracy,Precision,Recall,F1-score,F2-score,file size,latency
CNN-Mel,218.0,0.99,0.98,0.98,0.98,0.98,30.125 Kilobytes,1.318
SqueezeNet-Mel,117.0,0.99,0.96,1.0,0.98,0.99,807.539 Kilobytes,10.825
BNN-Mel,102.0,0.95,0.91,0.96,0.93,0.95,49.953 Kilobytes,1.604
Transformer-Mel,185.0,0.96,0.94,0.93,0.94,0.93,60.836 Kilobytes,2.528
CNN-Time,29.0,0.97,0.95,0.95,0.95,0.95,18.906 Kilobytes,3.157
SqueezeNet-Time,128.0,0.79,0.61,1.0,0.75,0.88,87.039 Kilobytes,15.626
BNN-Time,205.0,0.73,0.55,0.97,0.7,0.84,25.172 Kilobytes,4.157
Transformer-Time,19.0,0.87,0.81,0.79,0.8,0.8,29.352 Kilobytes,7.226
